In [1]:
import sys
print(sys.version)

3.9.7 (tags/v3.9.7:1016ef3, Aug 30 2021, 20:19:38) [MSC v.1929 64 bit (AMD64)]


In [2]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [3]:
import numpy as np
from flask import jsonify

import pandas as pd
import tensorflow as tf
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical  

In [4]:
data = pd.read_csv('goods_10K.csv')
data.head()

,name,category
0,Поло Print Bar,10
1,Футболка VseMayki.Ru,10
2,Майка Print Bar,10
3,"Friedrich Wilhelm Hermann Wagener ""Staats- un...",7
4,Шорты Quiksilver,10


In [5]:
labels = [
    'Авто',
    'Товары для здоровья',
    'Электроника',
    'Бытовая техника',
    'Строительство и ремонт',
    'Товары для дома',
    'Детские товары',
    'Досуг и развлечения',
    'Компьютерная техника',
    'Товары для красоты',
    'Одежда, обувь и аксессуары',
    'Продукты',
    'Спорт и отдых',
    'Дача, сад и огород',
    'Товары для животных',
]

In [6]:
X = data['name']
y = data['category']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y, shuffle=True)

Пока идем нормально

In [8]:
y_train.head()

5985    10
3641    10
6765    10
3578    10
5491    10
Name: category, dtype: int64

In [9]:
y_train_final = to_categorical(y_train)
y_test_final = to_categorical(y_test)

In [10]:
print(y_train_final[0:10])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [11]:
print(X_train.shape, y_train_final.shape)
print(X_test.shape, y_test_final.shape)

(7000,) (7000, 15)
(3000,) (3000, 15)


In [12]:
# Максимальное количество уникальных токетов (слов) в нашем словаре
MAX_WORDS = 50000
# Это специальное значение для слов, которых нет в нашем словаре
OOV_TOKEN = '<OOV>'
# Максимальная длина предложения
MAX_SEQUENCE_LENGTH = 10
EMBEDDING_DIM = 100
# Сколько раз будем прогонять весь тренировочный набор данных при обучении
EPOCHS = 3
BATCH_SIZE = 16

In [13]:
tokenizer = Tokenizer(
    num_words=MAX_WORDS, 
    filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~0123456789', # Фильтруем спецсимволы и цифры
    lower=True, 
    oov_token=OOV_TOKEN)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
print('Всего %s уникальных токенов' % len(word_index))

Всего 7540 уникальных токенов


In [14]:
def preprocess_text(texts):
    texts = tokenizer.texts_to_sequences(texts)
    return pad_sequences(texts, maxlen=MAX_SEQUENCE_LENGTH)

In [15]:
X_train_final = preprocess_text(X_train)
X_test_final = preprocess_text(X_test)
print(X_train_final.shape)
print(X_test_final.shape)

(7000, 10)
(3000, 10)


In [16]:
print(X_train_final[0:10])

[[   0    0    0    0    0    0    0    2    4    3]
 [   0    0    0    0    0    0    0    2    4    3]
 [   0    0    0    0    0    0    0   16    4    3]
 [   0    0    0    0    0    0    0    0   67 2200]
 [   0    0    0    0    0    0    0    8  835  517]
 [   0    0    0    0    0    0    0  211  638  212]
 [   0    0    0  134  124 2201 2202   71  164 2203]
 [   0    0    0    0    0 1244 2204   19 2205   93]
 [   0    0    0    0    0    0    0    7    4    3]
 [   0    0    0    0    0    0    0    0   34 2206]]


In [17]:
model = Sequential()
model.add(layers.Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(layers.SpatialDropout1D(0.2))
model.add(layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(len(labels), activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 100)           5000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 10, 100)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 15)                1515      
                                                                 
Total params: 5,081,915
Trainable params: 5,081,915
Non-trainable params: 0
_________________________________________________________________


In [18]:
print(len(model.layers))

4


In [19]:
model.fit(X_train_final, y_train_final, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.1, verbose=1)

Epoch 1/3
394/394 [==============================] - 17s 39ms/step - loss: 0.7441 - accuracy: 0.8113 - val_loss: 0.3747 - val_accuracy: 0.8929
Epoch 2/3
394/394 [==============================] - 14s 37ms/step - loss: 0.3122 - accuracy: 0.9038 - val_loss: 0.2976 - val_accuracy: 0.9100
Epoch 3/3
394/394 [==============================] - 14s 36ms/step - loss: 0.1575 - accuracy: 0.9519 - val_loss: 0.2557 - val_accuracy: 0.9486


In [20]:
def save_model(model, path):
    tf.keras.models.save_model(
        model,
        path,
        overwrite=True,
        include_optimizer=True,
        save_format=None)

In [21]:
import pickle

def save_tokenizer(tokenizer, path):
    with open(path, 'wb') as file:
        pickle.dump(tokenizer, file, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
save_model(model, "goods_classifier.h5")
save_tokenizer(tokenizer, "tokenizer.pickle")

In [23]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test_final, y_test_final, batch_size=32)
print("test loss, test acc:", results)

Evaluate on test data
94/94 [==============================] - 0s 1ms/step - loss: 0.2606 - accuracy: 0.9310
test loss, test acc: [0.2606315612792969, 0.9309999942779541]


In [24]:
    x = preprocess_text("Карта памяти Samsung")
    y = model.predict(x)
    result = []
    for item in y:
        index = np.argmax(item)
        result.append({
            "label": labels[index],
            "confidence": item.tolist()[index],
        })

    print(result)

[{'label': 'Одежда, обувь и аксессуары', 'confidence': 0.5731292366981506}, {'label': 'Одежда, обувь и аксессуары', 'confidence': 0.5783060789108276}, {'label': 'Детские товары', 'confidence': 0.5758166909217834}, {'label': 'Детские товары', 'confidence': 0.43981531262397766}, {'label': 'Одежда, обувь и аксессуары', 'confidence': 0.5783060789108276}, {'label': 'Одежда, обувь и аксессуары', 'confidence': 0.6860765814781189}, {'label': 'Детские товары', 'confidence': 0.5666932463645935}, {'label': 'Одежда, обувь и аксессуары', 'confidence': 0.5783060789108276}, {'label': 'Детские товары', 'confidence': 0.4702834188938141}, {'label': 'Детские товары', 'confidence': 0.6185156106948853}, {'label': 'Детские товары', 'confidence': 0.43981531262397766}, {'label': 'Одежда, обувь и аксессуары', 'confidence': 0.492744117975235}, {'label': 'Одежда, обувь и аксессуары', 'confidence': 0.6860765814781189}, {'label': 'Одежда, обувь и аксессуары', 'confidence': 0.822939932346344}, {'label': 'Одежда, об